# Study en fonction du nombre de neurones stables

In [1]:
import numpy as np 
import pandas as pd
from tools import get_project_path
import matplotlib.pyplot as plt

tab1 = pd.read_csv(get_project_path("results/benchmark/6x100-0.026/FINAL/SDPu/results-full-pruning-RLT=100.csv"))
tab1 = tab1[tab1['model']!='LP'].copy()
tab2 = pd.read_csv(get_project_path("results/benchmark/6x100-0.026/09_18_16h10_19s_SDPu-full-pruning-RLT=100-data-index=56-96-99/results.csv"))
# #tab1 = tab1[tab1["data_index"]<29].copy()
# tab2 = pd.read_csv(get_project_path("results/benchmark/9x100-0.026/09_18_15h42_46s_SDPu-full-pruning-RLT=100-28-56-96-97/results.csv"))
#tab3 = pd.read_csv(get_project_path("results/benchmark/9x200-0.015/09_23_08h54_49s_SDPt-RLT=0.2-55-99/results.csv"))
#tab4 = pd.read_csv(get_project_path("results/benchmark/9x200-0.015/11_05_23h21_56s_SDPt-RLT=0.2-35-44-84-99/results.csv"))  
# # # #tab5 = pd.read_csv(get_project_path("results/benchmark/9x100-0.026/FINAL/SDP-IP/results-60-99.csv"))
# # # # tab = pd.concat([tab1,tab2]µ)
# # #tab2 = tab2[tab2["data_index"]<79].copy();
tab = pd.concat([tab1, tab2]).reset_index(drop=True)

Nb_neurons = 9*200

# nb_stables_study = pd.read_csv(get_project_path("results/benchmark/6x200-0.015/FINAL/SDPu/results-full-pruning-RT=0.6.csv"))
# nb_stables_study["Stable"] = nb_stables_study["Number_actives_stable"] + nb_stables_study["Number_inactives_stable"]
# nb_stables_study["Unstable"] = Nb_neurons - nb_stables_study["Stable"]

In [ ]:
# tab = tab.drop(columns= ['Nb_stable_actives', 'Nb_stable_inactives'])
# tab['Stable'] = nb_stables_study['Stable']
# tab['Stable_prop'] = tab['Stable'] / Nb_neurons

In [ ]:
tab["data_index"].nunique()

In [ ]:
tab["Stable"] = tab["Nb_stable_actives"] + tab["Nb_stable_inactives"]
tab['Stable_prop'] = tab['Stable'] / Nb_neurons

In [ ]:
tab["Stable"].max()

In [ ]:
if "RLT_prop" in tab.columns:
    tab["RLT_prop"] = tab["RLT_prop"].fillna(0)
    RLT_prop = tab["RLT_prop"].unique()
    
    print("RLT_prop:", RLT_prop)
    #RLT_props = [0, 0.1, 0.3, 0.5]
    #tab = tab[tab["RLT_prop"].isin(RLT_props)].copy()

tab["total_time"] = tab["time"] + tab["bound_time"]
tab["robust"] = (tab["optimal_value"] >= 0) | (tab["status"]=="trivially_solved")
tab["target"] = tab["target"].fillna("unknown")

In [ ]:
tab

In [ ]:

robust_mean_data_index = tab.groupby(["data_index", "target"])["robust"].any()
#(robust_mean_data_index == 1).sum()
rob = robust_mean_data_index.reset_index().groupby("data_index")["robust"].all()
rob

In [ ]:
rob.sum()

In [ ]:
tab["total_time"].groupby(tab["data_index"]).sum().mean()

In [ ]:

tab["Unstable_prop"]= (Nb_neurons - (tab["Nb_stable_actives"] + tab["Nb_stable_inactives"])) / Nb_neurons

In [ ]:
tab["Unstable_prop"]

In [ ]:
tab = tab[tab['status']!='trivially_solved'].copy()
bins = np.arange(0, 1.1, 0.1)
tab['Stable_prop_bin'] = pd.cut(tab['Stable_prop'], bins=bins)

grouped = tab.groupby('Stable_prop_bin')['robust']
percentage = grouped.mean() * 100       # pourcentage de robust
counts = grouped.count()                # nombre d'éléments

# Plot
fig, ax = plt.subplots(figsize=(10,6))
bars = ax.bar(percentage.index.astype(str), percentage.values,
              color='skyblue', edgecolor='black')


# Labels
ax.set_ylabel('Pourcentage de données vérifiées robustes')
ax.set_xlabel('Pourcentage de neurones stables')
ax.set_title('Pourcentage de données vérifiées en fonction de la stabilité des neurones')
plt.xticks(rotation=45)
ax.legend()

# Ajouter le nombre d'éléments sur chaque barre
for bar, n in zip(bars, counts.values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 1,
            f'n={n}', ha='center', va='bottom', fontsize=9)


plt.tight_layout()
plt.show()

# Etude du gap quand on prune ou pas les neurones